In [1]:
import os
import pandas as pd
import numpy as np
import zipfile


#libraries for data visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
import seaborn as sns
#sns.set_palette('Set2')
sns.set_color_codes("pastel")

In [2]:
folder = "/Users/reejungkim/Documents/instacart-market-basket-analysis"

os.listdir(folder)

['order_products__prior.csv.zip',
 '.DS_Store',
 'order_products__train.csv.zip',
 'aisles.csv.zip',
 'orders.csv.zip',
 'departments.csv.zip',
 'products.csv.zip',
 'sample_submission.csv.zip']

In [3]:

for file in os.listdir(folder):
    if file.endswith(".zip"):
        zip_file = folder + "/"+ file
        zf = zipfile.ZipFile(zip_file)
        
        x = str(file[:-8])
        print("\n"+ x)
        globals()[x] = pd.read_csv(zf.open(file[:-4])) #, index_col=0)

        display(globals()[x])


order_products__prior


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1



order_products__train


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1



aisles


...

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk



products


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8



sample_submission


,order_id,products
0,17,39276 29259
1,34,39276 29259
2,137,39276 29259
3,182,39276 29259
4,257,39276 29259
...,...,...
74995,3420740,39276 29259
74996,3420877,39276 29259
74997,3420888,39276 29259
74998,3420989,39276 29259


In [4]:
order_products__prior.reordered.unique()#value_counts

array([1, 0])

In [5]:
df = order_products__prior.merge(orders, how='left', on='order_id')
df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


In [6]:
df = df.merge(products, how='left', on='product_id')
df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13


In [7]:
df.product_name.value_counts

<bound method IndexOpsMixin.value_counts of 0                                          Organic Egg Whites
1                                       Michigan Organic Kale
2                                               Garlic Powder
3                                              Coconut Butter
4                                           Natural Sweetener
                                  ...                        
32434484            Free & Clear Natural Dishwasher Detergent
32434485         Organic Mini Sandwich Crackers Peanut Butter
32434486                      All Natural French Toast Sticks
32434487                                               Banana
32434488    Organic  Sweet & Salty Peanut Pretzel Granola ...
Name: product_name, Length: 32434489, dtype: object>

In [8]:
df = df.merge(departments, how='left', on='department_id')
df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,department
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,dairy eggs
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,produce
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,pantry
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,pantry
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,pantry


In [9]:
df.pivot_table(index='department', values=['add_to_cart_order', 'reordered'],aggfunc=np.sum, margins=True )

,add_to_cart_order,reordered
department,,
alcohol,834315,87595
babies,4482098,245369
bakery,9513613,739188
beverages,18768220,1757892
breakfast,6474139,398013
bulk,290380,19950
canned goods,10624249,488535
dairy eggs,40580338,3627221
deli,9139364,638864


In [10]:
pd.DataFrame(df.groupby('order_id').department.agg(lambda x: x.tolist())).reset_index()


,order_id,department
0,2,"[dairy eggs, produce, pantry, pantry, pantry, ..."
1,3,"[dairy eggs, dairy eggs, produce, produce, dai..."
2,4,"[bakery, personal care, snacks, breakfast, bre..."
3,5,"[produce, pantry, deli, produce, dairy eggs, h..."
4,6,"[beverages, household, household]"
...,...,...
3214869,3421079,[personal care]
3214870,3421080,"[dairy eggs, frozen, beverages, produce, produ..."
3214871,3421081,"[snacks, bakery, pantry, dairy eggs, canned go..."
3214872,3421082,"[meat seafood, dairy eggs, produce, produce, d..."
